<a href="https://colab.research.google.com/github/akashbilgi/DMT/blob/main/Part2_from_scratch_DMT_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle


In [2]:
print("data read")
data = pd.read_csv('/content/drive/MyDrive/data.csv')

data read


In [3]:
data.drop(data.columns[[-1, 0]], axis=1, inplace=True)

In [4]:
# convert categorical labels to numbers
diag_map = {'M': 1.0, 'B': -1.0}
data['diagnosis'] = data['diagnosis'].map(diag_map)

In [5]:
Y = data.loc[:, 'diagnosis']
X = data.iloc[:, 1:]

In [6]:
def remove_correlated_features(X):
    corr_threshold = 0.9
    corr = X.corr()
    drop_columns = np.full(corr.shape[0], False, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i + 1, corr.shape[0]):
            if corr.iloc[i, j] >= corr_threshold:
                drop_columns[j] = True
    columns_dropped = X.columns[drop_columns]
    X.drop(columns_dropped, axis=1, inplace=True)
    return columns_dropped

In [7]:
def remove_less_significant_features(X, Y):
    sl = 0.05
    regression_ols = None
    columns_dropped = np.array([])
    for itr in range(0, len(X.columns)):
        regression_ols = sm.OLS(Y, X).fit()
        max_col = regression_ols.pvalues.idxmax()
        max_val = regression_ols.pvalues.max()
        if max_val > sl:
            X.drop(max_col, axis='columns', inplace=True)
            columns_dropped = np.append(columns_dropped, [max_col])
        else:
            break
    regression_ols.summary()
    return columns_dropped

In [8]:
remove_correlated_features(X)
remove_less_significant_features(X, Y)

array(['smoothness_mean', 'compactness_worst', 'compactness_mean',
       'radius_mean', 'texture_se', 'symmetry_se', 'smoothness_se',
       'concavity_worst'], dtype='<U32')

In [9]:
    X_normalized = MinMaxScaler().fit_transform(X.values)
    X = pd.DataFrame(X_normalized)

In [10]:
X.insert(loc=len(X.columns), column='intercept', value=1)

In [11]:
    print("splitting dataset into train and test sets...")
    X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=42)

splitting dataset into train and test sets...


In [12]:
def sgd(features, outputs):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)

        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is: {} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights

In [13]:
def calculate_cost_gradient(W, X_batch, Y_batch):
    # if only one example is passed (eg. in case of SGD)
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])  # gives multidimensional array

    distance = 1 - (Y_batch * np.dot(X_batch, W))
    # print(Y_batch,X_batch,W,distance)
    
    dw = np.zeros(len(W))

    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (regularization_strength * Y_batch[ind] * X_batch[ind])
        dw += di

    dw = dw/len(Y_batch)  # average
    return dw

In [14]:
def compute_cost(W, X, Y):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = regularization_strength * (np.sum(distances) / N)

    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost

In [15]:
regularization_strength = 10000
learning_rate = 0.000001

In [16]:
y_train.to_numpy().shape

(455,)

In [17]:
    print("training started...")
    W = sgd(X_train.to_numpy(), y_train.to_numpy())
    print("training finished.")
    print("weights are: {}".format(W))

training started...
Epoch is: 1 and Cost is: 7304.550623505597
Epoch is: 2 and Cost is: 6536.662604824531
Epoch is: 4 and Cost is: 5465.2109058005235
Epoch is: 8 and Cost is: 3833.912956023411
Epoch is: 16 and Cost is: 2651.3223645245785
Epoch is: 32 and Cost is: 1970.1600170850159
Epoch is: 64 and Cost is: 1588.7360086039039
Epoch is: 128 and Cost is: 1326.3303828987855
Epoch is: 256 and Cost is: 1159.3796210715184
Epoch is: 512 and Cost is: 1074.6236688103775
Epoch is: 1024 and Cost is: 1053.3165434925993
Epoch is: 2048 and Cost is: 1040.4917566368297
Epoch is: 4096 and Cost is: 1040.2073910656986
training finished.
weights are: [ 3.55482369 11.30824276 -2.2904084  -7.96498158 10.23704876 -1.20967837
 -7.24021306  2.10771417 -3.63985541  3.15467284  4.92299196  4.85910188
 -4.65895709]


In [18]:
print("testing the model...")
y_train_predicted = np.array([])
for i in range(X_train.shape[0]):
    yp = np.sign(np.dot(X_train.to_numpy()[i], W))
    y_train_predicted = np.append(y_train_predicted, yp)

y_test_predicted = np.array([])
for i in range(X_test.shape[0]):
    yp = np.sign(np.dot(X_test.to_numpy()[i], W))
    y_test_predicted = np.append(y_test_predicted, yp)


testing the model...


In [19]:
print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_test_predicted)))
print("recall on test dataset: {}".format(recall_score(y_test, y_test_predicted)))
print("precision on test dataset: {}".format(recall_score(y_test, y_test_predicted)))


accuracy on test dataset: 0.9912280701754386
recall on test dataset: 0.9767441860465116
precision on test dataset: 0.9767441860465116
